# 🤝 05: Multi-Agent HandOffs 실전 (LangGraph)

앞선 02·03·04 노트북에서 `StateGraph`와 `create_react_agent`로 **루프·라우팅·도구 사용**을 익혔어요.  
이번에는 그 기반 위에 "왜 멀티 에이전트 HandOffs가 프로덕션에서 필수인지"를 체감형 실습으로 정리합니다.


## 🧭 용어 매핑 (빠른 정리)
- **ReAct 루프**: 생각(Reason) → 행동(Act) → 관찰(Observe)로 자기 자신에게 되돌아오는 엣지
- **Swarm HandOffs**: 에이전트가 상태와 컨텍스트를 들고 다음 담당자에게 순차 인계
- **LangGraph StateGraph**: 노드·엣지를 명시적으로 선언해 결정론, 체크포인트, 가드, 병렬 처리를 통합


## 📈 Multi-Agent 진화 타임라인 (LangGraph 관점)
1. **Super-Agent (2022)**: 모든 도구·역할을 단일 에이전트에 몰아넣음 → 선택 피로, 비용↑
2. **ReAct & Planner-Executor (2022~2023)**: 루프·계획으로 질서↑ → 루프 제어·실패 복구가 취약
3. **병렬·위원회 패턴 (2023 중반)**: 품질↑ → 통합·조정 비용↑
4. **Swarm HandOffs (2023 후반~2024)**: 필요할 때만 적합한 에이전트에게 상태 + 컨텍스트 인계
5. **LangGraph StateGraph (2024~)**: 그래프/DAG 기반으로 체크포인트, HITL, 가드레일, 팬인·팬아웃까지 표준화


## 🚀 HandOffs가 해결하는 핵심 이슈
**Before (Super-Agent / 병렬)**
- 도구 선택 혼란, 순서 불명확, 평균적 결과

**After (HandOffs / StateGraph)**
- 라우터 → Research → Calculator → Writer 단계별 인계
- 상태에 작업 맥락과 결과를 저장하며 필요한 전문 에이전트만 활성화


## 🧪 오늘 실습 로드맵
- **Lab A**: ReAct 루프 한계를 직접 관찰하고 제어 장치를 붙여보기
- **Lab B**: Swarm HandOffs 스타일 미니 라우터 구현으로 상태 인계 감각 익히기
- **Lab C**: LangGraph StateGraph로 프로덕션급 멀티 에이전트 핸드오프 파이프라인 구축


## 📦 Colab & 오픈소스 환경 준비
Colab에서 바로 실행 가능하도록 모든 의존성을 오픈소스 패키지로 설치합니다. 출력은 `%%capture`로 숨겨둡니다.


In [ ]:
%%capture
!pip install -q --upgrade "transformers>=4.44.2" accelerate bitsandbytes     langgraph langchain langchain-core langchain-community langchain-huggingface langchain-teddynote     sentence-transformers faiss-cpu


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import math
from typing import Dict, List, TypedDict, Literal, Any, Optional
from uuid import uuid4

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command

from langchain_teddynote.messages import stream_graph


In [ ]:
print("🧠 Qwen2.5-7B-Instruct (4bit) 모델을 로드합니다...")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚙️ 사용 디바이스: {device}")

model_id = "Qwen/Qwen2.5-7B-Instruct"

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
)

text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.1,
    top_p=0.9,
)

llm = ChatHuggingFace(llm=HuggingFacePipeline(pipeline=text_gen))
print("✅ Qwen2.5-7B-Instruct 4bit 준비 완료! (한국어 + 툴콜 지원)")


In [ ]:
MARKET_DATA = {
    "size_2022": 120.0,
    "size_2023": 150.0,
    "size_2024": 184.7,
    "major_players": ["OpenAI", "Google", "Microsoft", "Amazon"],
    "trends": ["Generative AI", "Multimodal AI", "Enterprise AI"],
}

def format_market_report(summary: str, forecast: str) -> str:
    return (
        "요약: {summary}"
        "전망: {forecast}"
        "핵심 기업: {players}"
        "핵심 트렌드: {trends}"
    ).format(
        summary=summary,
        forecast=forecast,
        players=", ".join(MARKET_DATA["major_players"]),
        trends=", ".join(MARKET_DATA["trends"]),
    )


In [ ]:
@tool
def market_research_tool(topic: str) -> str:
    """간단한 시장 조사 결과를 반환합니다."""
    print("🔧 market_research_tool 호출")
    report = (
        f"AI 시장은 2024년에 {MARKET_DATA['size_2024']}B USD 규모로 성장했어요. "
        f"주요 기업은 {', '.join(MARKET_DATA['major_players'])}입니다."
    )
    return report

@tool
def growth_calculator_tool(series: str) -> str:
    """단순 성장률을 계산합니다. series는 쉼표로 구분된 숫자입니다."""
    print("🔧 growth_calculator_tool 호출")
    try:
        numbers = [float(x.strip()) for x in series.split(',')]
        if len(numbers) < 2:
            return "두 개 이상의 숫자가 필요합니다."
        start, end = numbers[0], numbers[-1]
        years = len(numbers) - 1
        cagr = (end / start) ** (1 / years) - 1
        return f"연평균 성장률(CAGR): {cagr * 100:.2f}%"
    except Exception as exc:
        return f"성장률 계산 오류: {exc}"

TOOLS = [market_research_tool, growth_calculator_tool]
MEMORY = MemorySaver()


# 1. Lab A – ReAct 루프 한계 체감하기
ReAct 패턴은 그 자체로 강력하지만, 복합 업무에서는 **루프 제어와 컨텍스트 관리**가 어려울 수 있습니다. 먼저 기본 에이전트를 실행하고 어떤 제약이 있는지 직접 확인해봅니다.


In [ ]:
REACT_POLICY = """
당신은 전문 리서치 에이전트입니다. 아래 형식을 반드시 지키세요.

Thought: 현재 생각을 한국어로 간단히 정리 (도구 호출 전 필요).
Action: 사용할 도구 이름만 입력 (예: market_research_tool).
Action Input: JSON 문자열 형태로 도구에 전달할 인자를 작성. 예: {"topic": "AI market"}
Observation: 도구가 반환한 결과를 요약.

도구를 더 이상 사용하지 않을 때만 Final Answer를 출력하세요.
Final Answer: 한국어 요약.

도구를 사용하지 않고 추측해서 답변하면 안 됩니다.
"""

react_agent = create_react_agent(
    llm,
    TOOLS,
    checkpointer=MEMORY,
    prompt=REACT_POLICY,
)

print("🧠 ReAct Agent 구성 완료!")

def run_react_demo(task: str, recursion_limit: int = 8) -> None:
    """ReAct 루프 실행 + 툴 사용 요약"""
    thread_id = f"lab-a-demo-{uuid4().hex[:4]}"
    config = {"configurable": {"thread_id": thread_id}, "recursion_limit": recursion_limit}

    print("🧪 Lab A 데모 실행")
    print("-" * 60)
    stream_graph(
        react_agent,
        {"messages": [HumanMessage(content=task)]},
        config=config,
    )
    print("-" * 60)

    records = MEMORY.get(config)
    if not records:
        print("⚠️ 저장된 상태가 없어 툴 사용 요약을 생성할 수 없습니다.")
        return
    latest_entry = records[-1]
    if isinstance(latest_entry, dict):
        latest_state = latest_entry.get("state", latest_entry)
    elif isinstance(latest_entry, (list, tuple)) and len(latest_entry) >= 2:
        latest_state = latest_entry[1]
    else:
        latest_state = latest_entry
    messages = latest_state.get("messages", [])

    tool_msgs = [msg for msg in messages if isinstance(msg, ToolMessage)]
    final_reply = next((msg.content for msg in reversed(messages) if isinstance(msg, AIMessage)), "")

    print("📊 툴 사용 요약")
    if tool_msgs:
        for msg in tool_msgs:
            print(f"- {msg.name}: {msg.content}")
    else:
        print("- 사용된 툴이 없습니다")
    print("
📝 최종 응답")
    print(final_reply.strip())

    if hasattr(MEMORY, "delete"):
        MEMORY.delete(config)

sample_task = (
    "반드시 다음 순서를 지켜서 답변하세요:
"
    "1. market_research_tool을 호출해 최신 시장 요약을 얻는다.
"
    "2. growth_calculator_tool을 호출해 성장률을 계산한다.
"
    "3. 두 도구 결과를 바탕으로 투자자용 요약을 한국어로 작성한다.
"
    "요청: AI 시장을 조사하고, 최근 성장률을 계산한 뒤, 투자자용 요약 보고서를 작성해줘."
)

run_react_demo(sample_task)


## 🔍 실행 결과 해석 가이드
- 출력 상단의 `stream_graph` 섹션에서 ReAct 루프가 어떻게 진행됐는지 확인하세요.
- `📊 툴 사용 요약`에 `market_research_tool`, `growth_calculator_tool`이 모두 나타나면 도구 호출이 정상 실행된 것입니다.
- `📝 최종 응답`은 도구 결과를 반영한 요약입니다. 이 섹션을 강의 슬라이드나 실습 안내에 바로 사용할 수 있습니다.


from dataclasses import dataclass

@dataclass
class ReactRunResult:
    content: str
    stop_reason: str
    steps: int
    tool_calls: List[str]


def run_react_with_controls(task: str, max_steps: int, stop_keyword: Optional[str] = None) -> ReactRunResult:
    """ReAct 실행을 통제하고 결과 요약을 반환합니다."""
    thread_id = f"lab-a-ctrl-{uuid4().hex[:4]}"
    config = {"configurable": {"thread_id": thread_id}, "recursion_limit": max_steps}

    list(react_agent.stream({"messages": [HumanMessage(content=task)]}, config=config))

    records = MEMORY.get(config)
    if not records:
        return ReactRunResult(content="", stop_reason="no_state", steps=0, tool_calls=[])

    latest_entry = records[-1]
    if isinstance(latest_entry, dict):
        latest_state = latest_entry.get("state", latest_entry)
    elif isinstance(latest_entry, (list, tuple)) and len(latest_entry) >= 2:
        latest_state = latest_entry[1]
    else:
        latest_state = latest_entry
    messages = latest_state.get("messages", [])

    tool_msgs = [msg for msg in messages if isinstance(msg, ToolMessage)]
    tool_calls = [msg.name for msg in tool_msgs]
    final_reply = next((msg.content for msg in reversed(messages) if isinstance(msg, AIMessage)), "")

    stop_reason = "finished"
    if stop_keyword and stop_keyword in final_reply:
        stop_reason = f"keyword:{stop_keyword}"
    elif len(tool_calls) >= max_steps:
        stop_reason = "max_steps"

    if hasattr(MEMORY, "delete"):
        MEMORY.delete(config)

    return ReactRunResult(
        content=final_reply.strip(),
        stop_reason=stop_reason,
        steps=len(tool_calls),
        tool_calls=tool_calls,
    )


In [ ]:
control_result = run_react_with_controls(
    task=sample_task,
    max_steps=3,
    stop_keyword="보고서",
)

print("🛑 종료 사유:", control_result.stop_reason)
print("🧮 사용한 툴 수:", control_result.steps)
print("🔧 툴 호출 목록:", control_result.tool_calls)
print("
📝 최종 응답")
print(control_result.content)


In [ ]:
from dataclasses import dataclass

@dataclass
class ReactRunResult:
    content: str
    stop_reason: str

# TODO: 미션 A-1 구현

def run_react_with_controls(task: str, max_steps: int, stop_keyword: str) -> ReactRunResult:
    raise NotImplementedError("미션 A-1을 완료하고 보고서를 제출하세요!")


> 💡 힌트: `stream_graph`를 다시 활용해 이벤트를 순회하면서 키워드를 감지하고, `recursion_limit`으로 최대 스텝을 제한할 수 있습니다.


# 2. Lab B – Swarm HandOffs 미니 라우터 만들기
Swarm 스타일에서는 각 에이전트가 **`next_agent`, `state`**를 반환하며 자연스럽게 바톤을 넘깁니다. 간단한 상태 사전을 사용해 컨텍스트가 어떻게 전달되는지 살펴봅니다.


In [ ]:
class HandOffState(TypedDict):
    current_agent: str
    messages: List[str]
    results: Dict[str, Any]
    next_action: str

def supervisor_agent(state: HandOffState) -> tuple[str, HandOffState]:
    state = state.copy()
    state["current_agent"] = "supervisor"
    state["next_action"] = "시장 조사 계획 수립"
    state["messages"].append("Supervisor: 우선 시장 데이터를 모아보자")
    return "research_agent", state

def research_agent(state: HandOffState) -> tuple[str, HandOffState]:
    state = state.copy()
    state["current_agent"] = "research_agent"
    state["next_action"] = "성장률 계산"
    state["results"]["research"] = market_research_tool.func("AI market")
    state["messages"].append("Research: 시장 조사 완료")
    return "calculator_agent", state

def calculator_agent(state: HandOffState) -> tuple[str, HandOffState]:
    state = state.copy()
    state["current_agent"] = "calculator_agent"
    values = ",".join(str(MARKET_DATA[k]) for k in ["size_2022", "size_2023", "size_2024"])
    state["results"]["growth"] = growth_calculator_tool.func(values)
    state["messages"].append("Calculator: 성장률 계산 완료")
    state["next_action"] = "보고서 작성"
    return "writer_agent", state

def writer_agent(state: HandOffState) -> tuple[str, HandOffState]:
    state = state.copy()
    state["current_agent"] = "writer_agent"
    summary = state["results"].get("research", "")
    forecast = state["results"].get("growth", "")
    state["results"]["report"] = format_market_report(summary, forecast)
    state["messages"].append("Writer: 최종 보고서 작성")
    state["next_action"] = "완료"
    return "END", state

HANDOFF_PIPELINE = {
    "supervisor": supervisor_agent,
    "research_agent": research_agent,
    "calculator_agent": calculator_agent,
    "writer_agent": writer_agent,
}

def run_swarm_handoff(initial_state: HandOffState) -> HandOffState:
    agent = "supervisor"
    state = initial_state
    while agent != "END":
        agent, state = HANDOFF_PIPELINE[agent](state)
    return state

initial_state: HandOffState = {
    "current_agent": "user",
    "messages": ["User: AI 시장 조사하고 성장률 계산해서 보고서 써줘"],
    "results": {},
    "next_action": ""
}

final_state = run_swarm_handoff(initial_state)
print(final_state["results"]["report"])


## 🔍 미션 B-1: 조건부 HandOff 추가
- `research_agent`에서 시장 데이터가 부족하다고 판단되면 `calculator_agent` 대신 `human_review`로 넘기도록 조건을 추가해보세요.
- 새로운 `human_review` 에이전트를 생성해 `messages`에 TODO를 남기고 다시 `writer_agent`로 연결되도록 설계합니다.


In [ ]:
# TODO: 미션 B-1 구현 공간
# 아래에 조건부 handoff 로직과 human_review 에이전트를 추가하세요.


# 3. Lab C – LangGraph StateGraph로 프로덕션 워크플로 구축
이제 LangGraph를 활용해 **결정론적**이고 **복구 가능한** HandOff 파이프라인을 구현합니다. `Command` 객체로 제어권을 명확히 넘기고, 체크포인트를 통해 어디서든 재시작이 가능합니다.


In [ ]:
class WorkflowState(TypedDict):
    messages: List[Any]
    current_agent: str
    results: Dict[str, Any]
    audit_log: List[str]

workflow_memory = MemorySaver()


In [ ]:
def router_node(state: WorkflowState) -> Command:
    latest = state["messages"][-1].content if state["messages"] else ""
    update = {
        "current_agent": "research_agent",
        "audit_log": [*state["audit_log"], f"Router → Research ({latest[:30]}...)"]
    }
    return Command(goto="research_agent", update=update)


def research_node(state: WorkflowState) -> Command:
    summary = market_research_tool.func("AI market")
    update = {
        "current_agent": "research_agent",
        "results": {**state["results"], "research": summary},
        "audit_log": [*state["audit_log"], "Research 완료"],
    }
    return Command(goto="calculator_agent", update=update)


def calculator_node(state: WorkflowState) -> Command:
    values = ",".join(str(MARKET_DATA[k]) for k in ["size_2022", "size_2023", "size_2024"])
    growth = growth_calculator_tool.func(values)
    update = {
        "current_agent": "calculator_agent",
        "results": {**state["results"], "growth": growth},
        "audit_log": [*state["audit_log"], "Calculator 완료"],
    }
    return Command(goto="writer_agent", update=update)


def writer_node(state: WorkflowState) -> Command:
    summary = state["results"].get("research", "")
    forecast = state["results"].get("growth", "")
    report = format_market_report(summary, forecast)
    update = {
        "current_agent": "writer_agent",
        "results": {**state["results"], "report": report},
        "audit_log": [*state["audit_log"], "Writer 완료"],
    }
    return Command(goto="quality_guard", update=update)


def quality_guard_node(state: WorkflowState) -> Command:
    report = state["results"].get("report", "")
    if "성장률" not in report:
        update = {
            "audit_log": [*state["audit_log"], "Guard: 계산 누락 → Calculator 재실행"],
        }
        return Command(goto="calculator_agent", update=update)
    update = {
        "audit_log": [*state["audit_log"], "Guard 통과"],
    }
    return Command(goto="writer_signoff", update=update)


def writer_signoff_node(state: WorkflowState) -> Command:
    update = {
        "audit_log": [*state["audit_log"], "Writer Sign-off"],
    }
    return Command(goto=END, update=update)


In [ ]:
workflow = StateGraph(WorkflowState)
workflow.add_node("router", router_node)
workflow.add_node("research_agent", research_node)
workflow.add_node("calculator_agent", calculator_node)
workflow.add_node("writer_agent", writer_node)
workflow.add_node("quality_guard", quality_guard_node)
workflow.add_node("writer_signoff", writer_signoff_node)

workflow.add_edge(START, "router")

langgraph_app = workflow.compile(checkpointer=workflow_memory)

print("✅ LangGraph 워크플로 컴파일 완료")


In [ ]:
workflow_input = {
    "messages": [HumanMessage(content="AI 시장 조사하고 성장률 계산해서 보고서 써줘")],
    "current_agent": "user",
    "results": {},
    "audit_log": [],
}

print("🧪 LangGraph 실행")
print("-" * 60)
stream_graph(
    langgraph_app,
    workflow_input,
    config={"configurable": {"thread_id": "lab-c"}},
)
print("-" * 60)
result = langgraph_app.invoke(workflow_input, config={"configurable": {"thread_id": "lab-c"}})
print(result["results"].get("report", "결과 없음"))
print("감사 로그:")
for log in result["audit_log"]:
    print(f"- {log}")


## 🔍 미션 C-1: HITL / 재시도 전략 추가
- `quality_guard_node`가 조건을 만족하지 못했을 때 `human_review` 노드로 분기하도록 확장해보세요.
- `workflow.compile`에 `interrupt_before=["human_review"]` 옵션을 주고, 사용자 입력으로 승인/수정을 받는 흐름을 만들어봅니다.
- 추가로, `calculator_node` 실패 시 재시도 로직을 `Command`로 구현해 품질을 강화해보세요.


## 🧭 프레임워크별 구현 관점 요약
| 패턴 | 장점 | 한계 | 언제 쓰나 |
| --- | --- | --- | --- |
| ReAct (Lab A) | 빠른 PoC, 도구 루프 자동화 | 루프 제어·복구·병렬 처리 약함 | 단일 에이전트 + 소규모 태스크 |
| Swarm HandOffs (Lab B) | 상태 인계 명확, 경량 라우팅 | 감사 로그·재시도·팬인 설계 직접 구현 필요 | 작은 팀/간단한 분기 |
| LangGraph StateGraph (Lab C) | 결정론, 체크포인트, 가드, 병렬/재시도 내장 | 초기 설계 비용, 그래프 정의 필요 | 프로덕션/규모 있는 자동화 |


## ✅ 마무리 체크포인트
- [ ] ReAct 루프의 최대 스텝·종료 조건을 직접 제어했다.
- [ ] Swarm HandOffs에서 상태와 컨텍스트 인계 흐름을 구현했다.
- [ ] LangGraph `Command`와 체크포인트로 프로덕션 워크플로를 실행했다.


## 🚀 다음 단계 제안
1. LangGraph 그래프에 병렬 조(팬아웃) 노드를 추가해 보고서를 이미지/숫자 버전으로 분기해보세요.
2. 실시간 데이터 소스를 연결할 MCP 또는 외부 API 에이전트를 붙여보세요.
3. `workflow_memory` 대신 Redis나 Postgres 체크포인트를 사용해 장기 실행 시나리오를 실험해보세요.
